# IMAGE CLASSIFIER

We are going to build an image classifier that when given a picture, can classify whether it is an image or a fish

## Dataset

We are going to use [imagenet](https://www.image-net.org/) a database with over hundred of thousands of images. It contains more than 14 million images and 20,000 image categories. It’s the standard that all image classifiers judge themselves against